In [9]:
import os  # 导入操作系统模块，用于文件路径操作
import json  # 导入JSON模块，用于处理JSON数据
import requests  # 导入HTTP请求库，用于API调用
import numpy as np  # 导入NumPy库，用于数值计算
from typing import List, Dict, Any, Optional  # 导入类型提示
from dataclasses import dataclass  # 导入数据类装饰器
import pickle  # 导入pickle模块，用于序列化对象
import re  # 导入正则表达式模块
from pathlib import Path  # 导入Path类，用于路径操作

# 使用 OpenAI embeddings 或 Hugging Face transformers 作为替代
try:
    from sentence_transformers import SentenceTransformer  # 尝试导入sentence_transformers库
    USE_SENTENCE_TRANSFORMERS = True  # 设置标志表示可以使用sentence_transformers
except ImportError:
    USE_SENTENCE_TRANSFORMERS = False  # 设置标志表示不能使用sentence_transformers
    # 备用方案：使用 Hugging Face transformers
    try:
        from transformers import AutoTokenizer, AutoModel  # 尝试导入transformers库
        import torch  # 导入PyTorch库
        USE_TRANSFORMERS = True  # 设置标志表示可以使用transformers
    except ImportError:
        USE_TRANSFORMERS = False  # 设置标志表示不能使用transformers

import faiss  # 导入FAISS库，用于向量检索

@dataclass
class Document:
    """文档数据结构"""
    id: str  # 文档唯一标识符
    content: str  # 文档内容
    metadata: Dict[str, Any] = None  # 文档元数据，默认为None
    embedding: Optional[np.ndarray] = None  # 文档的嵌入向量，默认为None

class DeepSeekClient:
    """DeepSeek API客户端,向DeepSeek API发送请求"""
    
    def __init__(self, api_key: str, base_url: str = "https://api.deepseek.com"):
        self.api_key = api_key  # 存储API密钥
        self.base_url = base_url  # 存储API基础URL
        self.headers = {  # 设置HTTP请求头
            "Authorization": f"Bearer {api_key}",  # 添加认证信息
            "Content-Type": "application/json"  # 设置内容类型
        }
    
    def chat_completion(self, messages: List[Dict[str, str]], 
                       model: str = "deepseek-chat", 
                       max_tokens: int = 2048,
                       temperature: float = 0.7) -> str:
        """调用DeepSeek聊天完成API"""
        url = f"{self.base_url}/chat/completions"  # 构建API端点URL
        
        payload = {  # 构建请求负载
            "model": model,  # 指定模型
            "messages": messages,  # 设置消息列表
            "max_tokens": max_tokens,  # 设置最大生成token数
            "temperature": temperature,  # 设置温度参数
            "stream": False  # 不使用流式响应
        }
        
        try:
            response = requests.post(url, headers=self.headers, json=payload)  # 发送POST请求
            response.raise_for_status()  # 检查HTTP响应状态
            
            result = response.json()  # 解析JSON响应
            return result["choices"][0]["message"]["content"]  # 返回生成的内容
        
        except requests.exceptions.RequestException as e:
            print(f"API请求错误: {e}")  # 打印API请求错误
            return ""  # 返回空字符串
        except KeyError as e:
            print(f"响应解析错误: {e}")  # 打印响应解析错误
            return ""  # 返回空字符串

class DocumentProcessor:
    """文档处理器,将文档分割成块"""
    
    def __init__(self, chunk_size: int = 512, chunk_overlap: int = 50):
        self.chunk_size = chunk_size  # 设置文本块大小
        self.chunk_overlap = chunk_overlap  # 设置文本块重叠大小
    
    def load_text_file(self, file_path: str) -> str:
        """加载文本文件"""
        with open(file_path, 'r', encoding='utf-8') as f:  # 打开文件
            return f.read()  # 读取文件内容
    
    def split_text(self, text: str) -> List[str]:
        """将文本分割成块"""
        # 按段落分割
        paragraphs = re.split(r'\n\s*\n', text)  # 使用正则表达式按段落分割文本
        
        chunks = []  # 初始化文本块列表
        current_chunk = ""  # 初始化当前文本块
        
        for paragraph in paragraphs:  # 遍历每个段落
            # 如果当前块加上新段落超过限制，保存当前块
            if len(current_chunk) + len(paragraph) > self.chunk_size:  # 检查是否超过块大小限制
                if current_chunk:  # 如果当前块不为空
                    chunks.append(current_chunk.strip())  # 添加当前块到列表
                current_chunk = paragraph  # 开始新的块
            else:
                current_chunk += "\n\n" + paragraph if current_chunk else paragraph  # 将段落添加到当前块
        
        # 添加最后一块
        if current_chunk:  # 如果最后一块不为空
            chunks.append(current_chunk.strip())  # 添加到列表
        
        return chunks  # 返回文本块列表
    
    def process_file(self, file_path: str) -> List[Document]:
        """处理单个文件"""
        text = self.load_text_file(file_path)  # 加载文件内容
        chunks = self.split_text(text)  # 分割文本
        
        documents = []  # 初始化文档列表
        filename = Path(file_path).name  # 获取文件名
        
        for i, chunk in enumerate(chunks):  # 遍历每个文本块
            doc = Document(  # 创建文档对象
                id=f"{filename}_{i}",  # 设置文档ID
                content=chunk,  # 设置文档内容
                metadata={  # 设置文档元数据
                    "source": filename,  # 源文件名
                    "chunk_id": i,  # 块ID
                    "file_path": file_path  # 文件路径
                }
            )
            documents.append(doc)  # 添加文档到列表
        
        return documents  # 返回文档列表

class EmbeddingModel:
    """嵌入模型包装器，主要作用是把文本变为稠密向量"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        all-MiniLM-L6-v2模型尺寸
        """
        self.model_name = model_name  # 存储模型名称
        self.model = None  # 初始化模型为None
        self.tokenizer = None  # 初始化分词器为None
        self._initialize_model()  # 初始化模型
    
    def _initialize_model(self):
        """初始化嵌入模型"""
        if USE_SENTENCE_TRANSFORMERS:  # 如果可以使用sentence_transformers
            try:
                self.model = SentenceTransformer(self.model_name)  # 加载sentence_transformers模型
                self.model_type = "sentence_transformers"  # 设置模型类型
                print(f"使用 sentence-transformers 模型: {self.model_name}")  # 打印使用的模型
            except Exception as e:
                print(f"sentence-transformers 初始化失败: {e}")  # 打印初始化失败信息
                self._fallback_to_transformers()  # 回退到transformers
        else:
            self._fallback_to_transformers()  # 回退到transformers
    
    def _fallback_to_transformers(self):
        """回退到 transformers 库"""
        if USE_TRANSFORMERS:  # 如果可以使用transformers
            try:
                # 使用更简单的模型
                model_name = "sentence-transformers/all-MiniLM-L6-v2"  # 设置模型名称
                self.tokenizer = AutoTokenizer.from_pretrained(model_name)  # 加载分词器
                self.model = AutoModel.from_pretrained(model_name)  # 加载模型
                self.model_type = "transformers"  # 设置模型类型
                print(f"使用 transformers 模型: {model_name}")  # 打印使用的模型
            except Exception as e:
                print(f"transformers 初始化失败: {e}")  # 打印初始化失败信息
                self._fallback_to_simple()  # 回退到简单方法
        else:
            self._fallback_to_simple()  # 回退到简单方法
    
    def _fallback_to_simple(self):
        """回退到简单的嵌入方法"""
        print("回退到简单的 TF-IDF 嵌入方法")  # 打印回退信息
        self.model_type = "tfidf"  # 设置模型类型
        try:
            from sklearn.feature_extraction.text import TfidfVectorizer  # 导入TF-IDF向量化器
            self.model = TfidfVectorizer(max_features=384, stop_words='english')  # 创建TF-IDF向量化器
            self.fitted = False  # 设置未拟合标志
        except ImportError:
            print("警告: sklearn 未安装，使用随机嵌入")  # 打印警告
            self.model_type = "random"  # 设置模型类型为随机
    
    def encode(self, texts: List[str], show_progress_bar: bool = True) -> np.ndarray:
        """编码文本为嵌入向量"""
        if self.model_type == "sentence_transformers":  # 如果使用sentence_transformers
            return self.model.encode(texts, show_progress_bar=show_progress_bar)  # 使用模型编码文本
        
        elif self.model_type == "transformers":  # 如果使用transformers
            return self._encode_with_transformers(texts)  # 使用transformers编码
        
        elif self.model_type == "tfidf":  # 如果使用TF-IDF
            return self._encode_with_tfidf(texts)  # 使用TF-IDF编码
        
        else:  # random
            return self._encode_random(texts)  # 使用随机编码
    
    def _encode_with_transformers(self, texts: List[str]) -> np.ndarray:
        """使用 transformers 编码"""
        embeddings = []  # 初始化嵌入列表
        
        for text in texts:  # 遍历每个文本
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)  # 对文本进行分词
            
            with torch.no_grad():  # 不计算梯度
                outputs = self.model(**inputs)  # 获取模型输出
                # 使用 [CLS] token 的嵌入或平均池化
                embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # 计算嵌入向量
                embeddings.append(embedding)  # 添加到列表
        
        return np.array(embeddings)  # 返回嵌入向量数组
    
    def _encode_with_tfidf(self, texts: List[str]) -> np.ndarray:
        """使用 TF-IDF 编码"""
        if not self.fitted:  # 如果模型未拟合
            self.model.fit(texts)  # 拟合模型
            self.fitted = True  # 设置已拟合标志
        
        return self.model.transform(texts).toarray()  # 转换文本并返回数组
    
    def _encode_random(self, texts: List[str]) -> np.ndarray:
        """随机嵌入（仅用于测试）"""
        print("警告: 使用随机嵌入，结果可能不准确")  # 打印警告
        return np.random.randn(len(texts), 384)  # 返回随机嵌入向量

class VectorStore:
    """向量存储和检索，主要作用是存储和检索文本的向量表示"""
    
    def __init__(self, embedding_model_name: str = "all-MiniLM-L6-v2"):
        """
        初始化向量存储
        """
        self.embedding_model = EmbeddingModel(embedding_model_name)  # 创建嵌入模型
        self.documents: List[Document] = []  # 初始化文档列表
        self.index = None  # 初始化索引为None
        self.dimension = None  # 初始化维度为None
    
    def add_documents(self, documents: List[Document]):
        """添加文档到向量存储"""
        print(f"正在处理 {len(documents)} 个文档...")  # 打印处理信息
        
        # 生成嵌入向量
        contents = [doc.content for doc in documents]  # 提取文档内容
        embeddings = self.embedding_model.encode(contents, show_progress_bar=True)  # 编码文档内容
        
        # 添加嵌入向量到文档
        for doc, embedding in zip(documents, embeddings):  # 遍历文档和嵌入向量
            doc.embedding = embedding  # 设置文档的嵌入向量
        
        self.documents.extend(documents)  # 将文档添加到列表
        
        # 构建FAISS索引
        self._build_index()  # 构建索引
        
        print(f"成功添加 {len(documents)} 个文档")  # 打印成功信息
    
    def _build_index(self):
        """构建FAISS索引"""
        if not self.documents:  # 如果没有文档
            return  # 直接返回
        
        embeddings = np.array([doc.embedding for doc in self.documents])  # 获取所有嵌入向量
        self.dimension = embeddings.shape[1]  # 设置维度
        
        # 创建FAISS索引
        self.index = faiss.IndexFlatIP(self.dimension)  # 内积相似度
        
        # 标准化向量（用于余弦相似度）
        faiss.normalize_L2(embeddings)  # L2标准化
        self.index.add(embeddings.astype(np.float32))  # 添加向量到索引
    
    def search(self, query: str, top_k: int = 5) -> List[Document]:
        """搜索相关文档"""
        if not self.index:  # 如果索引不存在
            return []  # 返回空列表
        
        # 生成查询向量
        query_embedding = self.embedding_model.encode([query])  # 编码查询
        faiss.normalize_L2(query_embedding)  # L2标准化
        
        # 搜索
        scores, indices = self.index.search(query_embedding.astype(np.float32), top_k)  # 执行搜索
        
        # 返回相关文档
        results = []  # 初始化结果列表
        for score, idx in zip(scores[0], indices[0]):  # 遍历分数和索引
            if idx < len(self.documents):  # 如果索引有效
                doc = self.documents[idx]  # 获取文档
                results.append(doc)  # 添加到结果
        
        return results  # 返回结果
    
    def save(self, file_path: str):
        """保存向量存储"""
        data = {  # 创建数据字典
            'documents': self.documents,  # 存储文档
            'dimension': self.dimension  # 存储维度
        }
        
        with open(file_path, 'wb') as f:  # 打开文件
            pickle.dump(data, f)  # 序列化数据
        
        # 保存FAISS索引
        if self.index:  # 如果索引存在
            faiss.write_index(self.index, file_path + ".faiss")  # 写入索引
    
    def load(self, file_path: str):
        """加载向量存储"""
        with open(file_path, 'rb') as f:  # 打开文件
            data = pickle.load(f)  # 加载数据
        
        self.documents = data['documents']  # 设置文档
        self.dimension = data['dimension']  # 设置维度
        
        # 加载FAISS索引
        if os.path.exists(file_path + ".faiss"):  # 如果索引文件存在
            self.index = faiss.read_index(file_path + ".faiss")  # 读取索引

class RAGSystem:
    """RAG系统主类，主要作用是提供RAG系统的核心功能"""
    
    def __init__(self, deepseek_api_key: str, embedding_model: str = "all-MiniLM-L6-v2"):
        self.deepseek_client = DeepSeekClient(deepseek_api_key)  # 创建DeepSeek客户端
        self.document_processor = DocumentProcessor()  # 创建文档处理器
        self.vector_store = VectorStore(embedding_model)  # 创建向量存储
        self.conversation_history = []  # 初始化对话历史
    
    def add_documents_from_files(self, file_paths: List[str]):
        """从文件添加文档"""
        all_documents = []  # 初始化文档列表
        
        for file_path in file_paths:  # 遍历文件路径
            print(f"处理文件: {file_path}")  # 打印处理信息
            documents = self.document_processor.process_file(file_path)  # 处理文件
            all_documents.extend(documents)  # 添加文档到列表
        
        self.vector_store.add_documents(all_documents)  # 将文档添加到向量存储
    
    def add_documents_from_text(self, texts: List[str], metadata_list: List[Dict] = None):
        """从文本添加文档"""
        documents = []  # 初始化文档列表
        
        for i, text in enumerate(texts):  # 遍历文本
            chunks = self.document_processor.split_text(text)  # 分割文本
            
            for j, chunk in enumerate(chunks):  # 遍历文本块
                doc = Document(  # 创建文档
                    id=f"text_{i}_{j}",  # 设置ID
                    content=chunk,  # 设置内容
                    metadata=metadata_list[i] if metadata_list else {"source": f"text_{i}"}  # 设置元数据
                )
                documents.append(doc)  # 添加文档到列表
        
        self.vector_store.add_documents(documents)  # 将文档添加到向量存储
    
    def _build_context(self, relevant_docs: List[Document]) -> str:
        """构建上下文"""
        context_parts = []  # 初始化上下文部分列表
        
        for i, doc in enumerate(relevant_docs):  # 遍历相关文档
            context_parts.append(f"文档 {i+1}:")  # 添加文档标题
            context_parts.append(doc.content)  # 添加文档内容
            context_parts.append("")  # 空行分隔
        
        return "\n".join(context_parts)  # 连接上下文部分
    
    def _build_prompt(self, query: str, context: str) -> str:
        """构建提示词"""
        prompt = f"""你是一个专业的AI助手。请根据以下上下文信息回答用户的问题。

上下文信息：
{context}

用户问题：{query}

请基于提供的上下文信息给出准确、详细的回答。如果上下文信息不足以回答问题，请说明这一点。

回答："""  # 构建提示词模板
        
        return prompt  # 返回提示词
    
    def query(self, question: str, top_k: int = 5, use_history: bool = False) -> Dict[str, Any]:
        """查询RAG系统"""
        # 检索相关文档
        relevant_docs = self.vector_store.search(question, top_k)  # 搜索相关文档
        
        if not relevant_docs:  # 如果没有找到相关文档
            return {  # 返回错误信息
                "answer": "抱歉，我没有找到相关的文档来回答您的问题。",
                "sources": [],
                "context": ""
            }
        
        # 构建上下文
        context = self._build_context(relevant_docs)  # 构建上下文
        
        # 构建消息
        messages = []  # 初始化消息列表
        
        # 添加历史对话（如果需要）
        if use_history:  # 如果使用历史
            messages.extend(self.conversation_history)  # 添加对话历史
        
        # 添加当前查询
        prompt = self._build_prompt(question, context)  # 构建提示词
        messages.append({"role": "user", "content": prompt})  # 添加用户消息
        print(messages)
        # 调用DeepSeek API
        answer = self.deepseek_client.chat_completion(messages)  # 获取回答
        
        # 更新对话历史
        if use_history:  # 如果使用历史
            self.conversation_history.append({"role": "user", "content": question})  # 添加用户问题
            self.conversation_history.append({"role": "assistant", "content": answer})  # 添加助手回答
            
            # 限制历史长度
            if len(self.conversation_history) > 10:  # 如果历史超过10条
                self.conversation_history = self.conversation_history[-10:]  # 保留最近10条
        
        # 提取来源信息
        sources = []  # 初始化来源列表
        for doc in relevant_docs:  # 遍历相关文档
            source_info = {  # 创建来源信息
                "document_id": doc.id,  # 文档ID
                "content_preview": doc.content[:200] + "..." if len(doc.content) > 200 else doc.content,  # 内容预览
                "metadata": doc.metadata  # 元数据
            }
            sources.append(source_info)  # 添加来源信息
        
        return {  # 返回结果
            "answer": answer,  # 回答
            "sources": sources,  # 来源
            "context": context,  # 上下文
            "relevant_documents": len(relevant_docs)  # 相关文档数量
        }
    
    def clear_history(self):
        """清除对话历史"""
        self.conversation_history = []  # 重置对话历史
    
    def save_system(self, file_path: str):
        """保存RAG系统"""
        self.vector_store.save(file_path)  # 保存向量存储
        
        # 保存对话历史
        history_path = file_path + ".history"  # 设置历史文件路径
        with open(history_path, 'w', encoding='utf-8') as f:  # 打开文件
            json.dump(self.conversation_history, f, ensure_ascii=False, indent=2)  # 保存对话历史
    
    def load_system(self, file_path: str):
        """加载RAG系统"""
        self.vector_store.load(file_path)  # 加载向量存储
        
        # 加载对话历史
        history_path = file_path + ".history"  # 设置历史文件路径
        if os.path.exists(history_path):  # 如果历史文件存在
            with open(history_path, 'r', encoding='utf-8') as f:  # 打开文件
                self.conversation_history = json.load(f)  # 加载对话历史

def main():
    """主函数示例"""
    # 设置API密钥
    DEEPSEEK_API_KEY = "sk-8d043c487cf04418bbdcf659f052331e"  # 请替换为你的实际API密钥
    
    # 创建RAG系统
    rag = RAGSystem(DEEPSEEK_API_KEY)  # 初始化RAG系统
    
    # 示例2: 从文本添加文档
    sample_texts = [  # 示例文本列表
        """
        人工智能（AI）是计算机科学的一个分支，致力于创建能够模拟人类智能的系统。
        AI系统可以学习、推理、解决问题和理解语言。机器学习是AI的一个重要子领域，
        它使计算机能够从数据中学习而无需明确编程。深度学习是机器学习的一个子集，
        使用神经网络来模拟人脑的工作方式。
        """,
        """
        自然语言处理（NLP）是人工智能的一个重要分支，专注于让计算机理解、
        解释和生成人类语言。NLP的应用包括机器翻译、情感分析、问答系统和
        文本摘要。现代NLP系统大多基于深度学习模型，如Transformer架构。
        """,
        """
        计算机视觉是人工智能的另一个重要领域，致力于让计算机能够识别和理解
        图像和视频内容。常见的计算机视觉任务包括图像分类、目标检测、
        图像分割和人脸识别。卷积神经网络（CNN）是计算机视觉中最常用的模型。
        """
    ]
    
    # 添加文档到系统
    rag.add_documents_from_text(sample_texts)  # 添加示例文本到RAG系统
    
    # 交互式查询
    print("=== DeepSeek RAG 系统 ===")  # 打印系统标题
    print("输入问题进行查询，输入 'quit' 退出")  # 打印使用说明
    print("输入 'clear' 清除对话历史")  # 打印清除历史说明
    print("输入 'save' 保存系统状态")  # 打印保存系统说明
    print("-" * 50)  # 打印分隔线
    
    while True:  # 无限循环
        question = input("\n请输入您的问题: ").strip()  # 获取用户输入
        
        if question.lower() == 'quit':  # 如果输入quit
            break  # 退出循环
        elif question.lower() == 'clear':  # 如果输入clear
            rag.clear_history()  # 清除历史
            print("对话历史已清除")  # 打印确认信息
            continue  # 继续循环
        elif question.lower() == 'save':  # 如果输入save
            rag.save_system("rag_system.pkl")  # 保存系统
            print("系统状态已保存")  # 打印确认信息
            continue  # 继续循环
        elif not question:  # 如果输入为空
            continue  # 继续循环
        
        # 查询
        print("正在查询...")  # 打印查询状态
        result = rag.query(question, use_history=True)  # 执行查询
        
        # 显示结果
        print(f"\n回答: {result['answer']}")  # 打印回答
        print(f"\n参考了 {result['relevant_documents']} 个相关文档")  # 打印文档数量
        
        # 显示来源（可选）
        show_sources = input("\n是否显示来源信息? (y/n): ").lower() == 'y'  # 询问是否显示来源
        if show_sources:  # 如果用户选择显示
            print("\n来源信息:")  # 打印标题
            for i, source in enumerate(result['sources']):  # 遍历来源
                print(f"{i+1}. 文档ID: {source['document_id']}")  # 打印文档ID
                print(f"   内容预览: {source['content_preview']}")
    print("=== DeepSeek RAG 系统 ===")
    print("输入问题进行查询，输入 'quit' 退出")
    print("输入 'clear' 清除对话历史")
    print("输入 'save' 保存系统状态")
    print("-" * 50)
    
    

if __name__ == "__main__":
    main()

No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

C:\Users\HDS\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HDS\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling b

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'role': 'user', 'content': '什么是AI'}, {'role': 'assistant', 'content': '根据提供的上下文信息，人工智能（AI）是计算机科学的一个分支，致力于创建能够模拟人类智能的系统。AI系统具备以下核心能力：\n\n1. **基础定义**（文档1）：\n- 可执行学习、推理、解决问题和理解语言等类人智能任务\n- 通过机器学习（尤其是深度学习）实现数据驱动的自主优化\n\n2. **关键技术分支**：\n- **计算机视觉**（文档2/4/5）：\n  - 使计算机能识别和理解视觉数据（图像/视频）\n  - 典型任务：图像分类、目标检测、人脸识别\n  - 主要采用卷积神经网络（CNN）\n\n- **自然语言处理NLP**（文档3）：\n  - 实现语言理解、生成与交互\n  - 应用包括机器翻译、情感分析等\n  - 依赖Transformer等深度学习架构\n\n3. **实现方式**（文档1）：\n- 机器学习：无需显式编程即可从数据中学习\n- 深度学习：通过神经网络模拟人脑处理机制\n\n注：上下文存在重复内容（文档2/4/5内容完全重复），但已涵盖AI的核心定义、关键子领域及实现方法，信息完整度足以回答该问题。'}, {'role': 'user', 'content': '心情不好怎么办'}, {'role': 'assistant', 'content': '根据提供的上下文信息，所有文档均聚焦于人工智能的技术领域（如计算机视觉、自然语言处理等），并未包含与情绪调节、心理健康相关的任何内容。因此，**当前上下文信息不足以回答“心情不好怎么办”这一问题**。\n\n建议：若需要相关帮助，可考虑以下通用建议（非基于当前上下文）：\n1. 与亲友倾诉\n2. 进行适度运动\n3. 尝试正念冥想\n4. 寻求专业心理咨询\n\n（注：作为AI助手，如需心理健康支持，建议优先联系人类专业人士。）'}, {'role': 'user', 'content': '心情不好怎么办'}, {'role': 'assistant', 'content': '根据提供的上下文信息，所有文档均聚焦于**人工智能的技术领域**（如计算机视觉、自然语言处理、机器学习等），**未包含任何与情绪调节、心理健康或生活建议

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'role': 'user', 'content': '心情不好怎么办'}, {'role': 'assistant', 'content': '根据提供的上下文信息，所有文档均聚焦于人工智能的技术领域（如计算机视觉、自然语言处理等），并未包含与情绪调节、心理健康相关的任何内容。因此，**当前上下文信息不足以回答“心情不好怎么办”这一问题**。\n\n建议：若需要相关帮助，可考虑以下通用建议（非基于当前上下文）：\n1. 与亲友倾诉\n2. 进行适度运动\n3. 尝试正念冥想\n4. 寻求专业心理咨询\n\n（注：作为AI助手，如需心理健康支持，建议优先联系人类专业人士。）'}, {'role': 'user', 'content': '心情不好怎么办'}, {'role': 'assistant', 'content': '根据提供的上下文信息，所有文档均聚焦于**人工智能的技术领域**（如计算机视觉、自然语言处理、机器学习等），**未包含任何与情绪调节、心理健康或生活建议相关的内容**。因此，**基于当前上下文无法给出与“心情不好怎么办”相关的专业回答**。  \n\n（注：作为AI助手，若您需要心理健康支持，建议参考可靠的心理学资源或联系专业心理咨询师。当前技术文档库仅涵盖AI技术定义及实现方法，不涉及人文关怀类内容。）'}, {'role': 'user', 'content': 'AI发展前景'}, {'role': 'assistant', 'content': '基于提供的上下文信息，AI的发展前景主要体现在以下技术领域和应用方向：\n\n### 1. **核心技术持续突破**\n- **机器学习与深度学习**（文档1）：  \n  作为AI的核心驱动力，无需显式编程的自主学习能力将持续增强，神经网络架构（尤其是模拟人脑的深度学习模型）将向更高效、更可解释的方向演进。\n- **计算机视觉**（文档2/4/5）：  \n  以CNN为代表的模型已在图像/视频理解领域成熟应用，未来可能在实时性（如自动驾驶）、细粒度识别（如医疗影像分析）及多模态融合（结合语音、文本等）方向突破。\n- **自然语言处理（NLP）**（文档3）：  \n  Transformer架构推动的生成式AI（如大语言模型）将进一步提升语言理解与生成的拟人化水平，扩展至更复杂的交

KeyboardInterrupt: 

In [ ]:
16*7.5

120.0

In [ ]:
15+15+12*7.5/64

31.40625

In [ ]:
15+14*7.5/64

16.640625

In [ ]:
16*7.5/64

1.875